# 2b: Compute Bleck energy cycle

This notebook assumes that `filter_data.ipynb` has been run with `bleck = True`

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
from xgcm import Grid
import dask.array as dsa
from dask.diagnostics import ProgressBar

In [2]:
#deg=4
deg=32

# Filter Specs
shape = 'Gaussian'
#shape = 'Taper'
coarsen_fac = 2
filter_fac = 32
compl = 'simple'

end_time = 1100

doSmallerSet = 1

In [ ]:
rho_ref = 1000

## Open filtered Neverworld2 model data

In [3]:
if deg == 4:
    run = 'nw2_0.25deg_N15_baseline'
    chunks = {'time': 20}
    nr_days = 500

if deg == 8:
    run = 'nw2_0.125deg_N15_baseline'
    chunks = {'time': 10}
    nr_days = 500

elif deg == 16:
    run = 'nw2_0.0625deg_N15_baseline'
    chunks = {'time': 5}
    nr_days = 500

elif deg == 32:
    #run = 'nw2_0.03125deg_N15_baseline_diags'
    run = 'nw2_0.03125deg_N15_baseline_hmix20'
    chunks = {'time': 1, 'zl':1}
    nr_days = 100

elif deg == 325:
    run = 'nw2_0.03125deg_N15_baseline_hmix5'
    chunks = {'time': 1, 'zl':1}
    nr_days = 100  
    
elif deg == 320:
    run = 'nw2_0.03125deg_N15_QG_Leith_v3'
    chunks = {'time': 1, 'zl':1}
    nr_days = 100

#path = '/glade/p/univ/unyu0004/gmarques/NeverWorld2' 
path = '/glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/'

st = xr.open_dataset('%s/%s/static.nc' % (path,run), decode_times=False)
av = xr.open_dataset('%s/%s/averages_%08d.nc' % (path, run, end_time-nr_days+2), decode_times=False, chunks=chunks)
sn = xr.open_dataset('%s/%s/snapshots_%08d.nc' % (path,run,end_time-nr_days+5), decode_times=False, chunks=chunks)

ffile_pref = '/glade/scratch/noraloose/filtered_data/' 
#ffile_pref = '/glade/campaign/cgd/oce/people/noraloose/filtered_data/'

# filtered snapshots

sn_f_h = xr.open_zarr('%s/%s/snapshots_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_h' % (ffile_pref, run, end_time-nr_days+5, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
sn_f_u = xr.open_zarr('%s/%s/snapshots_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_u' % (ffile_pref, run, end_time-nr_days+5, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
sn_f_v = xr.open_zarr('%s/%s/snapshots_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_v' % (ffile_pref, run, end_time-nr_days+5, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
sn_f = xr.merge([sn_f_h, sn_f_u, sn_f_v])
    
# filtered averages
av_f_h = xr.open_zarr('%s/%s/averages_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_h' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f_u = xr.open_zarr('%s/%s/averages_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_u' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f_v = xr.open_zarr('%s/%s/averages_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_v' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f_TWA = xr.open_zarr('%s/%s/averages_%08d_TWA_filtered_%s_coarse%i_fac%i_only_diffusion_%s' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f = xr.merge([av_f_h, av_f_u, av_f_v, av_f_TWA])
    
str_filter = 'diff'

## Prepare NW2 grid information

In [5]:
Nx = np.size(st.xh)
Ny = np.size(st.yh)

coords = {'X': {'center': 'xh', 'outer': 'xq'},
            'Y': {'center': 'yh', 'outer': 'yq'},
            'Z': {'center': 'zl', 'outer': 'zi'} }
grid = Grid(st, coords=coords, periodic=['X'])

st['dxT'] = grid.interp(st.dxCu,'X')
st['dyT'] = grid.interp(st.dyCv,'Y')
st['dxBu'] = grid.interp(st.dxCv,'X')
st['dyBu'] = grid.interp(st.dyCu,'Y',boundary='fill')

metrics = {('X',):['dxCu','dxCv','dxT','dxBu'],
           ('Y',):['dyCu','dyCv','dyT','dyBu'],
           ('X', 'Y'): ['area_t', 'area_u', 'area_v']
          }
grid = Grid(st, coords=coords, periodic=['X'], metrics=metrics)

In [6]:
ds = xr.Dataset() # new xarray dataset for EKE budget (to be saved to file)

ds.attrs['filter_specs'] = '(shape, factor, degree high-res run) = (%s %i, %i)' % (shape, filter_fac, deg)
for fld in ['time','zl','yh','xh']:
    ds[fld] = av_f[fld]

In [7]:
ffile = ffile_pref + run + '/EKE_TWA_%08d_%s_coarse%i_fac%i_%s' % (end_time-nr_days+2, 
                                                               shape, coarsen_fac, filter_fac, compl)
ffile

'/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline_diags/EKE_TWA_00001002_Gaussian_coarse2_fac32_simple'

# EKE budget

### EKE
$$
   \text{EKE}_{ij} = \overline{\texttt{h}\cdot \texttt{KE}}_{ij} 
   - \underbrace{\frac{1}{4} \frac{(\overline{\texttt{uh}}_{Ij}/\texttt{dyCu}_{Ij})^2 + (\overline{\texttt{uh}}_{I-1j}/\texttt{dyCu}_{I-1j})^2 
   +(\overline{\texttt{vh}}_{iJ}/\texttt{dxCv}_{iJ})^2 + (\overline{\texttt{vh}}_{iJ-1}/\texttt{dxCv}_{iJ-1})^2 }{\overline{\texttt{h}}_{ij}}}_{\texttt{MKE}_{ij}}
$$  

In [8]:
# volume-preserving
ufld = (av_f['uh']/st['dyCu'])**2
tfld1 = grid.interp(ufld.fillna(value=0),'X') 
vfld = (av_f['vh']/st['dxCv'])**2
tfld2 = grid.interp(vfld.fillna(value=0),'Y') 
ds['MKE_vol'] = 0.5 * (tfld1 + tfld2) / av_f['h']

data = av_f['hKE'] - ds['MKE_vol']
ds['EKE_vol'] = data.copy() 
#ds['EKE_vol'].attrs = {'units' : av_f['hKE'].units, 'long_name': 'Eddy Kinetic Energy (TWA)'}

### EKE Tendency

\begin{align*}
\text{EKE tendency}_{ij}& = \overline{\texttt{dKE\_dt}}_{ij} - \frac{d}{dt}\texttt{MKE}_{ij},
\end{align*}
where 
$$
    \frac{d}{dt}\texttt{MKE}_{ij} = \text{time-average}\left(\partial_t \texttt{MKE}_{ij}\right) = \frac{1}{\tau_1-\tau_0} \int_{\tau_0}^{\tau_1} \partial_t \texttt{MKE}_{ij} dt = \frac{\texttt{MKE}_{ij}(\tau_1) - \texttt{MKE}_{ij}(\tau_0)}{\tau_1-\tau_0}.
$$
is computed from snapshots.

In [9]:
if np.all(av_f.average_DT == av_f.average_DT[0]):
    deltat = av_f.average_DT[0] * 24 * 60 * 60
else: 
    raise AssertionError('averaging intervals vary')

In [12]:
if not doSmallerSet:
    doStep0 = 0
    # test
    hKE = sn_f['hKE']
    if np.array_equal(av_f.time_bnds[:,1],sn_f.time):
        hKEdt = (hKE - hKE.shift(time=1))/deltat
        hKEdt['time'] = av_f['h'].time
    else: 
        raise AssertionError('av and sn datasets not compatitble')

    hKEdt = hKEdt.where(st.wet)
    hKEdt = hKEdt.where(av_f.time>av_f.time[0])  

    # volume-preserving
    ufld = (sn_f['uh']/st['dyCu'])**2
    tfld1 = grid.interp(ufld.fillna(value=0),'X') 
    vfld = (sn_f['vh']/st['dxCv'])**2
    tfld2 = grid.interp(vfld.fillna(value=0),'Y') 
    MKE =  0.5 * (tfld1 + tfld2)/sn_f['h']


    if np.array_equal(av_f.time_bnds[:,1],sn_f.time):
        dMKEdt = (MKE - MKE.shift(time=1))/deltat
        dMKEdt['time'] = av_f['h'].time
    else: 
        raise AssertionError('av and sn datasets not compatitble')

    dMKE_dt = dMKEdt.where(st.wet)
    ds['dMKEdt_sn_vol'] = dMKEdt.copy()
    ds['dMKEdt_sn_vol'] = ds['dMKEdt_sn_vol'].where(av_f.time>av_f.time[0])  
    #ds['dMKEdt_sn_vol'].attrs = {'units' : av_f['dKE_dt'].units, 'long_name': 'MKE tendency (TWA), computed from snapshots'}

    ds['dEKEdt_sn_vol'] =  hKEdt - ds['dMKEdt_sn_vol']

    #ds['dEKEdt_sn_vol'].attrs = {'units' : av_f['dKE_dt'].units, 
    #                            'long_name': 'Eddy Kinetic Energy tendency (TWA), computed from snapshots'}

### EKE transport


\begin{align*}
    \text{EKE Transport} = \overline{\texttt{KE\_adv}} 
    +\frac{1}{\texttt{area\_t}} \Big\{& \delta_i\Big[ (\texttt{MKE}/\overline{\texttt{h}})_{T\to U} \underbrace{\overline{\texttt{uh}}}_\text{contains metric term}\Big]
 +  \delta_j
   \Big[ (\texttt{MKE}/\overline{\texttt{h}})_{T\to V}\overline{\texttt{vh}}\Big]
   \Big\},
\end{align*}
where
\begin{align*}
    ((\texttt{MKE}/\overline{\texttt{h}})_{T\to U})_{Ij} & =\frac{1}{2}\left(\texttt{MKE}_{ij}/\overline{\texttt{h}}_{ij}+\texttt{MKE}_{i+1j}/\overline{\texttt{h}}_{i+1j}\right)\\
      ((\texttt{MKE}/\overline{\texttt{h}}))_{T\to V})_{iJ} & = \frac{1}{2}\left(\texttt{MKE}_{ij}/\overline{\texttt{h}}_{ij}+\texttt{MKE}_{ij+1}/\overline{\texttt{h}}_{ij+1}\right) 
\end{align*}

are simple mid-point averages (analogous to definition of \texttt{KE}, \texttt{MKE}.

The transport (for both non-TWA and TWA) that stems from the pressure gradient term is:

$$
PG_\text{Transport} =-\frac{1}{\rho_1}\left( \sum_{n=1}^N \overline{\nabla\cdot(h_n\mathbf{u}_n p_n)}-\nabla\cdot(\overline{h_n\mathbf{u}_n} \overline{p_n})\right)
$$ 

In [13]:
if 'eta' not in av_f:
        h_rev = av_f['h'].isel(zl=slice(None,None,-1))
        eta = grid.cumsum(h_rev,'Z',boundary='fill')
        eta = eta.isel(zi=slice(None,None,-1))
        eta = eta + av_f['etaB']
        eta = eta.chunk({'zi':16})

        eta = xr.where(st.wet,eta,np.nan)
        eta = eta.transpose('time', 'zi', 'yh', 'xh') # reorder coordinates

        av_f['eta'] = eta.copy()
        av_f['eta'].attrs['long_name'] = 'Interface height'

In [14]:

if 'p' not in av_f:
        gprime = 10 * grid.diff(av_f.zl,'Z',boundary='fill')/rho_ref
        gprime[15] = np.nan

        pressure = grid.cumsum(gprime*av_f['eta'],'Z')
        pressure = rho_ref * pressure
        pressure = xr.where(st.wet,pressure,np.nan)
        pressure = pressure.transpose('time', 'zl', 'yh', 'xh') # reorder coordinates
        av_f['p'] = pressure.copy()
        av_f['p'].attrs = {'units' : 'kg s-2', 'long_name': 'Pressure'}

In [16]:
# PG Transport term
if not doSmallerSet:
    uflux = av_f['uh'] * grid.interp(av_f['p'].fillna(value=0),'X',metric_weighted=['X','Y'])
    uflux = grid.diff(uflux.fillna(value=0),'X')
    vflux = av_f['vh'] * grid.interp(av_f['p'].fillna(value=0),'Y',metric_weighted=['X','Y'],boundary='fill')
    vflux = grid.diff(vflux.fillna(value=0),'Y')
    data2 = -(uflux + vflux)/st.area_t/rho_ref
    ds['MKE_div'] = data2.where(st.wet)
    ds['MKE_div'] = ds['MKE_div'].chunk({'yh':Ny, 'xh':Nx})

    ds['Transport_PG'] = av_f['uhp_div'] - ds['MKE_div'] # uhp_div = - 1/rho_1 * \nabla\cdot(uhp)

############
# volume-preserving
ufld = (av_f['uh']/st['dyCu'])**2
tfld1 = grid.interp(ufld.fillna(value=0),'X') 
vfld = (av_f['vh']/st['dxCv'])**2
tfld2 = grid.interp(vfld.fillna(value=0),'Y') 
MKE =  0.5 * (tfld1 + tfld2)/av_f['h']
tfld = MKE/av_f['h']

ufld2 = grid.interp(tfld.fillna(value=0),'X') * av_f['uh']
vfld2 = grid.interp(tfld.fillna(value=0),'Y',boundary='fill') * av_f['vh']
term1 = grid.diff(ufld2.fillna(value=0),'X')
term2 = grid.diff(vfld2.fillna(value=0),'Y')

transport = av_f['KE_adv'] + term1/st.area_t + term2/st.area_t
transport = transport.chunk({'yh':Ny, 'xh':Nx})

if not doSmallerSet:
    ds['Transport_vol'] = (transport + ds['Transport_PG']).where(st.wet)

#ds['Transport_vol'].attrs = {'units' : ds['dEKEdt_sn_vol'].units, 
#                                'long_name': 'EKE Transport (TWA)'}
ds['Transport_no_PG_vol'] = transport.where(st.wet)
#ds['Transport_no_PG_vol'].attrs = {'units' : ds['dEKEdt_sn_vol'].units, 
#                                'long_name': 'EKE Transport without PG contribution (TWA)'}


### Exchange of KE with large scales


We define
\begin{align}
(\text{ke}_u)_{Ij} & = 
-\overline{\texttt{uh}}_{Ij}\cdot \texttt{dxCu}_{Ij}\cdot (\overline{\texttt{h\_rvxv}}_{Ij} + \overline{\texttt{h\_gKEu}}_{Ij})\\
(\text{ke}_v)_{iJ} & = -
\overline{\texttt{vh}}_{iJ}\cdot \texttt{dyCv}_{iJ}\cdot (\overline{\texttt{h\_rvxu}}_{iJ} + \overline{\texttt{h\_gKEv}}_{iJ})
\end{align}

\begin{align}
(\text{ke2}_u)_{Ij} & = 
\overline{\texttt{uh}}_{Ij}\cdot \texttt{dxCu}_{Ij}\cdot\left(-\frac{d}{dt}\overline{\texttt{uh}}_{Ij}/ \texttt{dyCu}_{Ij} + \overline{\texttt{h\_dudt}}_{Ij}\right)\\
(\text{ke2}_v)_{iJ} & = 
\overline{\texttt{vh}}_{iJ}\cdot \texttt{dyCv}_{iJ}\cdot \left(-\frac{d}{dt}\overline{\texttt{vh}}_{iJ}/ \texttt{dxCv}_{iJ} + \overline{\texttt{h\_dvdt}}_{iJ}\right)
\end{align}
and 
$$
    \frac{d}{dt}\texttt{uh}_{Ij} = \text{time-average}\left(\partial_t \texttt{uh}_{Ij}\right) = \frac{1}{\tau_1-\tau_0} \int_{\tau_0}^{\tau_1} \partial_t \texttt{uh}_{Ij} dt = \frac{\texttt{uh}_{Ij}(\tau_1) - \texttt{uh}_{Ij}(\tau_0)}{\tau_1-\tau_0}.
$$

$$
    \frac{d}{dt}\texttt{vh}_{iJ} = \text{time-average}\left(\partial_t \texttt{vh}_{iJ}\right) = \frac{1}{\tau_1-\tau_0} \int_{\tau_0}^{\tau_1} \partial_t \texttt{vh}_{iJ} dt = \frac{\texttt{vh}_{iJ}(\tau_1) - \texttt{vh}_{iJ}(\tau_0)}{\tau_1-\tau_0}.
$$

\begin{align*}
 (\text{KE exchange})_{ij} & = \frac{(\text{ke}_u)_{I-1j}+(\text{ke}_u)_{Ij}+(\text{ke}_v)_{iJ-1}+(\text{ke}_v)_{iJ}}{2\overline{\texttt{h}}_{ij}\texttt{area\_T}_{ij}}\\
& + \frac{(\text{ke2}_u)_{I-1j}+(\text{ke2}_u)_{Ij}+(\text{ke2}_v)_{iJ-1}+(\text{ke2}_v)_{iJ}}{2\overline{\texttt{h}}_{ij}\texttt{area\_T}_{ij}}\\
& + \texttt{MKE}_{ij}/\overline{\texttt{h}_{ij}}\cdot  \overline{\texttt{dhdt}_{ij}}\\
   &-\frac{1}{\texttt{area\_t}} \Big\{ \delta_i\Big[ (\texttt{MKE}/\overline{\texttt{h}})_{T\to U} \underbrace{\overline{\texttt{uh}}}_\text{contains metric term}\Big]
 +  \delta_j
   \Big[ (\texttt{MKE}/\overline{\texttt{h}})_{T\to V}\overline{\texttt{vh}}\Big]
   \Big\},
\end{align*}

In [17]:
# volume-preserving
ke_u = - av_f['uh'] * st['dxCu'] * (av_f['h_rvxv'] + av_f['h_gKEu'])
ke_v = - av_f['vh'] * st['dyCv'] * (av_f['h_rvxu'] + av_f['h_gKEv'])   

term1 = grid.interp(ke_u.fillna(value=0),'X') 
term1 = term1 + grid.interp(ke_v.fillna(value=0),'Y')  
term1 = term1/av_f['h']/st['area_t']

ke2_u = av_f['uh'] * st['dxCu'] * (av_f['h_du_dt'] - av_f['duhdt']/st['dyCu'])
ke2_v = av_f['vh'] * st['dyCv'] * (av_f['h_dv_dt'] - av_f['dvhdt']/st['dxCv'])

term2 = grid.interp(ke2_u.fillna(value=0),'X') 
term2 = term2 + grid.interp(ke2_v.fillna(value=0),'Y')  
term2 = term2/av_f['h']/st['area_t']

# discretize like MKE
ufld = (av_f['uh']/st['dyCu'])**2
tfld1 = grid.interp(ufld.fillna(value=0),'X') 
vfld = (av_f['vh']/st['dxCv'])**2
tfld2 = grid.interp(vfld.fillna(value=0),'Y') 
MKE =  0.5 * (tfld1 + tfld2)/av_f['h']
tfld = MKE/av_f['h']
term3 = tfld * av_f['dhdt']

ufld2 = grid.interp(tfld.fillna(value=0),'X') * av_f['uh']
vfld2 = grid.interp(tfld.fillna(value=0),'Y',boundary='fill') * av_f['vh']
term4 = grid.diff(ufld2.fillna(value=0),'X') + grid.diff(vfld2.fillna(value=0),'Y')
term4 = - term4/st['area_t']

data = term1 + term2 + term3 + term4
data = xr.where(st.wet,data,np.nan)

ds['Exchange_vol'] = data.copy()
ds['Exchange_vol'] = ds['Exchange_vol'].chunk({'yh':Ny, 'xh':Nx})

#ds['Exchange_vol'].attrs = {'units' : ds['dEKEdt_sn_vol'].units, 
#                               'long_name': 'KE Exchange (TWA)'}

KeyError: 'dhdt'

### Coriolis term


\begin{align*}
\text{Coriolis}_\text{vol} & = \overline{KECorAdv−KEadvA}- \overline{u}^V \cdot \overline{h\cdot (\texttt{CAu}-\texttt{rvxv}-\texttt{gKEu})_{U\to T}} - \overline{v}^V \cdot \overline{h\cdot (\texttt{CAv}-\texttt{rvxu}-\texttt{gKEv})_{V\to T}}
\end{align*}

In [18]:
# area-preserving
#ufld = av_f['u'] * st['area_u'] * (av_f['CAu'] - av_f['rvxv'] - av_f['gKEu'])
#vfld = av_f['v'] * st['area_v'] * (av_f['CAv'] - av_f['rvxu'] - av_f['gKEv'])

#term1 = - av_f['h'] * grid.interp(ufld.fillna(value=0),'X') 
#term2 = - av_f['h'] * grid.interp(vfld.fillna(value=0),'Y') 
    
#data = av_f['KE_CorAdv'] - av_f['KE_adv'] + (term1 + term2)/st['area_t']
#data = xr.where(st.wet,data,np.nan)
#ds['Coriolis_area'] = data.copy()
#ds['Coriolis_area'].attrs = {'units' : ds['dEKEdt_area'].units, 
#                                'long_name': 'Coriolis term (non-TWA)'}
  
# volume-preserving
ufld = av_f['uh'] * st['dxCu'] * (av_f['h_CAu'] - av_f['h_rvxv'] - av_f['h_gKEu'])
vfld = av_f['vh'] * st['dyCv'] * (av_f['h_CAv'] - av_f['h_rvxu'] - av_f['h_gKEv'])   
    
term1 = - grid.interp(ufld.fillna(value=0),'X')
term2 = - grid.interp(vfld.fillna(value=0),'Y')
    
data = av_f['KE_CorAdv'] - av_f['KE_adv'] + (term1 + term2)/(av_f['h'] * st['area_t'])
data = xr.where(st.wet,data,np.nan)    
ds['Coriolis_vol'] = data.copy()
#ds['Coriolis_vol'].attrs = {'units' : ds['dEKEdt_area'].units, 
#                                'long_name': 'Coriolis term (TWA)'}

### EKE Sources

$$
\text{Sources}_\text{vol}= \overline{PE\_to\_KE} + \overline{KE\_BT} - \frac{\overline{hu_{U\to T}}}{\overline{h}}\cdot \overline{h\cdot (PFu + u\_BT\_accel)_{U\to T}}- \frac{\overline{hv_{V\to T}}}{\overline{h}}\cdot \overline{h\cdot (PFv + v\_BT\_accel)_{V\to T}}.
$$


\begin{align}
    \sum_n (\bar{S}_n - \hat{S}_n)&=-\frac{1}{\rho_1}\left(\sum_{n=1}^N  \overline{h_n\mathbf{u}_n\cdot\nabla p_n}^A-  \sum_{n=1}^N  \overline{\mathbf{u}}^V_n\cdot\overline{h_n \nabla p_n}^A\right)\\
    &= \underbrace{-\frac{1}{\rho_1}\left(\sum_{n=1}^N\overline{h_n\mathbf{u}_n\cdot\nabla p_n}^A -\sum_{n=1}^N\overline{h_n\mathbf{u}_n}^A\cdot\overline{\nabla p_n}^A\right)}_\text{Transport + EPE tendency + Residual due to commuting derivative and filter}+ \underbrace{\frac{1}{\rho_1}\sum_{n=1}^N \overline{h_n\mathbf{u}_n}^A\cdot \overline{\nabla p_n'}^V}_\text{Eddy form stress}
    \end{align}



where 


$$
\frac{1}{\rho_1} \overline{h_n\mathbf{u}_n}^A\cdot \overline{\nabla p_n'}^V = \frac{1}{\rho_1}\left(\sum_{n=1}^N\overline{h_n\mathbf{u}_n}^A\cdot(\overline{\nabla p_n}^V - \overline{\nabla p_n}^A)\right)
$$

In [20]:
if not doSmallerSet:
    ufld = av_f['uh'] * st['dxCu']  * (av_f['PFu+u_BT_accel_visc_rem'])
    tfld1 = grid.interp(ufld.fillna(value=0),'X') 
    vfld = av_f['vh'] * st['dyCv'] * (av_f['PFv+v_BT_accel_visc_rem'])
    tfld2 = grid.interp(vfld.fillna(value=0),'Y',boundary='fill') 
    data3 = (tfld1 + tfld2) / st['area_t'] 
    data3 = xr.where(st.wet,data3,np.nan)

# volume-preserving
ufld = av_f['uh'] * st['dxCu'] * (av_f['h_PFu+u_BT_accel'])
vfld = av_f['vh'] * st['dyCv'] * (av_f['h_PFv+v_BT_accel'])   
    
term1 = - grid.interp(ufld.fillna(value=0),'X') / av_f['h'] / st['area_t']
term2 = - grid.interp(vfld.fillna(value=0),'Y') / av_f['h'] / st['area_t']
    
data = av_f['PE_to_KE+KE_BT'] + term1 + term2 
data = xr.where(st.wet,data,np.nan)    
  
ds['Sources_vol'] = data.copy()
#ds['Sources_vol'].attrs = {'units' : av_f['PE_to_KE'].units, 
#                             'long_name': 'EKE source term (TWA)'}

if not doSmallerSet:
    ds['eddy_form_vol'] =  (data3 + term1 + term2).where(st.wet)
    #ds['eddy_form_vol'].attrs = {'units' : av_f['PE_to_KE'].units, 
    #                             'long_name': 'Work done by eddy from stress (TWA)'}

### EKE Sinks

$$
\text{Sinks}_\text{vol}= \overline{KE\_horvisc}+\overline{KE\_visc}- \frac{\overline{hu_{U\to T}}}{\overline{h}}\cdot \overline{h\cdot (diffu + du\_dt\_visc)_{U\to T}}- \frac{\overline{hv_{V\to T}}}{\overline{h}} \cdot \overline{h\cdot (diffv + dv\_dt\_visc)_{V\to T}}.
$$

In [21]:
# volume-preserving
ufldA = av_f['uh'] * st['dxCu'] * av_f['h_diffu']
vfldA = av_f['vh'] * st['dyCv'] * av_f['h_diffv']
ufldB = av_f['uh'] * st['dxCu'] * av_f['h_du_dt_visc']
vfldB = av_f['vh'] * st['dyCv'] * av_f['h_dv_dt_visc'] 
ufldB_wind = av_f['uh'] * st['dxCu'] * av_f['h_du_dt_str']
vfldB_wind = av_f['vh'] * st['dyCv'] * av_f['h_dv_dt_str']

termuA = - grid.interp(ufldA.fillna(value=0),'X')/av_f['h']/st['area_t']
termuB = - grid.interp(ufldB.fillna(value=0),'X')/av_f['h']/st['area_t']
termuB_wind = - grid.interp(ufldB_wind.fillna(value=0),'X')/av_f['h']/st['area_t']

termvA = - grid.interp(vfldA.fillna(value=0),'Y')/av_f['h']/st['area_t']
termvB = - grid.interp(vfldB.fillna(value=0),'Y')/av_f['h']/st['area_t']
termvB_wind = - grid.interp(vfldB_wind.fillna(value=0),'Y')/av_f['h']/st['area_t']

dataA = av_f['KE_horvisc'] + termuA + termvA
dataA = xr.where(st.wet, dataA,np.nan)    

dataB = av_f['KE_visc'] + termuB + termvB
dataB = xr.where(st.wet,dataB,np.nan)    

dataB_wind = av_f['KE_stress'] + termuB_wind + termvB_wind
dataB_wind = xr.where(st.wet,dataB_wind,np.nan)  

ds['Sinks_hor_vol'] = dataA.copy()
ds['Sinks_vert_vol'] = dataB.copy()
ds['Sinks_vert_wind_vol'] = dataB_wind.copy()

ds['Sinks_vol'] = dataA + dataB


ds['MKE_sources_hor_vol'] = - termuA - termvA 
ds['MKE_sources_vert_vol'] = - termuB - termvB
ds['MKE_sources_vert_wind_vol'] = - termuB_wind - termvB_wind

## MPE --> MKE
$$
    -\frac{1}{\rho_1}\sum_{n=1}^N\overline{h_n\mathbf{u}_n}\cdot\nabla \overline{p_n}
$$

In [22]:
if not doSmallerSet:
    ke_u = av_f['uh'] * st['dxCu'] * av_f['PFu+u_BT_accel_visc_rem']
    ke_v = av_f['vh'] * st['dyCv'] * av_f['PFv+v_BT_accel_visc_rem']

    ds['MPE2MKE_vol'] = (
        grid.interp(ke_u.fillna(value=0),'X') 
        + grid.interp(ke_v.fillna(value=0),'Y')  
    ) / st['area_t']

## Save EKE budget terms to netcdf

In [23]:
ds = ds.transpose('time', 'zl', 'yh', 'xh') # reorder coordinates

In [24]:
ds2 = xr.Dataset() 
if doSmallerSet:
    fldlist = ['EKE_vol', 'Transport_no_PG_vol','Exchange_vol','Coriolis_vol',
           'Sources_vol','Sinks_hor_vol','Sinks_vert_vol', 'Sinks_vert_wind_vol']
    
else:
    fldlist = ['EKE_vol', 'dEKEdt_sn_vol','Transport_no_PG_vol','Exchange_vol','Coriolis_vol',
           'Sources_vol','Sinks_hor_vol','Sinks_vert_vol', 'Sinks_vert_wind_vol',
           'eddy_form_vol',
           'MPE2MKE_vol','MKE_sources_hor_vol','MKE_sources_vert_vol',
           'MKE_sources_vert_wind_vol',
           #'Transport_PG',
           'MKE_vol']
for fld in fldlist:
    ds2[fld] = ds[fld]
ds2

KeyError: 'Exchange_vol'

In [20]:
ffile

'/glade/scratch/noraloose/filtered_data/nw2_0.03125deg_N15_baseline/EKE_TWA_00000602_Gaussian_coarse2_fac32_simple'

In [21]:
ds2.to_zarr(ffile)